In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_diagonal_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:



cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) <100:
            X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,1888079.57,0.70,5.79,-7.39,0.02,-0.16,-0.02
4,0.0,0.70,1052609.25,-10.48,-5.87,2.73,0.14,0.02
5,0.0,5.79,-10.48,175510.94,0.64,0.92,0.35,-0.02
6,0.0,-7.39,-5.87,0.64,53930.97,0.04,-0.01,-0.00
7,0.0,0.02,2.73,0.92,0.04,6244.79,0.02,0.00
8,0.0,-0.16,0.14,0.35,-0.01,0.02,254.71,0.00
9,0.0,-0.02,0.02,-0.02,-0.00,0.00,0.00,5.92


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_20366/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,0.00000,0.00001,-0.00002,0.00000,-0.00001,-0.00001
4,NaN,0.00000,1.00000,-0.00002,-0.00002,0.00003,0.00001,0.00001
5,NaN,0.00001,-0.00002,1.00000,0.00001,0.00003,0.00005,-0.00002
6,NaN,-0.00002,-0.00002,0.00001,1.00000,0.00000,-0.00000,-0.00001
7,NaN,0.00000,0.00003,0.00003,0.00000,1.00000,0.00001,0.00002
8,NaN,-0.00001,0.00001,0.00005,-0.00000,0.00001,1.00000,0.00001
9,NaN,-0.00001,0.00001,-0.00002,-0.00001,0.00002,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

21.988309010408777

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.49727681e+11 1.34581364e+11 3.67888674e+09 3.31275481e+08
 4.17518763e+06 6.17847793e+03 2.50450421e+00 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.0,0.993495,-0.113143,-0.012342,-0.003764,-0.000435,-0.000018,-4.227662e-07
1,-0.0,0.112692,0.993131,-0.030163,-0.008996,-0.001035,-0.000042,-9.594166e-07
2,-0.0,0.015309,0.027927,0.997169,-0.067757,-0.006965,-0.000282,-6.725293e-06
3,0.0,0.005782,0.010386,0.067172,0.997223,-0.029861,-0.001198,-2.795474e-05
4,-0.0,0.000828,0.001483,0.008913,0.029290,0.999436,-0.013656,-3.162466e-04
5,0.0,0.000045,0.000081,0.000482,0.001574,0.013607,0.999838,-1.163306e-02
6,-0.0,0.000002,0.000003,0.000017,0.000055,0.000474,0.011628,9.999323e-01
7,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.8873080978715636,
 0.9720728009177446,
 0.9328283495907724,
 0.9707100827552648,
 0.9863928657808033,
 0.9883723833065505,
 1.0]